 Retriever and chain - RAG pipeline
 What is chain and retreiver is done for advanced RAG

What is chain ?

What is Retreiver ?



In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"C:/Users/LENOVO/Documents/Saranya/GitHUb/LANGCHAIN/rag/DL_Paper.pdf")
docs = loader.load()

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 20)
text_splitter.split_documents(docs)[:20]

[Document(page_content='A Distributional Perspective on Reinforcement Learning\nMarc G. Bellemare* 1Will Dabney* 1R´emi Munos1\nAbstract\nIn this paper we argue for the fundamental impor-\ntance of the value distribution : the distribution\nof the random return received by a reinforcement\nlearning agent. This is in contrast to the com-\nmon approach to reinforcement learning which\nmodels the expectation of this return, or value .\nAlthough there is an established body of liter-\nature studying the value distribution, thus far it\nhas always been used for a speciﬁc purpose such\nas implementing risk-aware behaviour. We begin\nwith theoretical results in both the policy eval-\nuation and control settings, exposing a signiﬁ-\ncant distributional instability in the latter. We\nthen use the distributional perspective to design\na new algorithm which applies Bellman’s equa-\ntion to the learning of approximate value distri-\nbutions. We evaluate our algorithm using the\nsuite of games from

In [16]:
# Faiss vector database
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs[:20] , OpenAIEmbeddings())

In [17]:
from langchain_community.llms import Ollama
## Load Ollama
llm =Ollama(model='llama2')
llm

Ollama()

In [7]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [18]:
## create chain
## using create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm , prompt)

In [19]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A16C292E70>)

In [20]:
# combine chain ( llm + prompt ) + Retreiver (faiss db as the retreiver )

"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [21]:
response=retrieval_chain.invoke({"input":"What is the Bellman’s equation"})

In [22]:
response['answer']

"Bellman's equation is a mathematical equation that describes the relationship between the value of a decision-making problem and the expected outcome of that decision. In reinforcement learning, Bellman's equation states that the value of a state-action pair is equal to the reward received plus the expected value of the next state-action pair, multiplied by a discount factor:\n\nQ(x,a) = ER(x,a) + γQ(X',A')\n\nwhere Q(x,a) is the value of the current state-action pair (x,a), ER(x,a) is the reward received, and γ is the discount factor. The equation states that the value of a decision is determined by the expected reward it receives plus the expected value of its next action.\n\nIn a broader sense, Bellman's equation can be applied to any decision-making problem, where the goal is to find the optimal policy that maximizes the expected value of the outcome. The equation can be used to model situations where there are multiple possible actions in a given state, and the goal is to determi